In [1]:
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

from bokeh.plotting import figure, show, save, ColumnDataSource
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool

In [3]:
trainlines = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='railway')

In [4]:
clyde = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='river')

In [5]:
stations = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='stations')

In [7]:
stations['name'] = stations['name'].str.upper()

In [6]:
subway  = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='subway')

In [23]:
subway['name'] = subway['name'].str.upper()

In [9]:
#tsource = ColumnDataSource(trainlines)
#stsource = ColumnDataSource(stations)
#ssource = ColumnDataSource(subway)

In [10]:
# update CRS for all layers (as shp were in BNG )
trainlines.crs = {'init' :'epsg:27700'} 
print('Trainlines: ', trainlines.crs)

#stations.crs = {'init' :'epsg:27700'} 
print('Stations : ', stations.crs)

#subway.crs = {'init' :'epsg:27700'}  
print('Subway : ',subway.crs)

#clyde.crs= {'init' :'epsg:27700'} 
print('Clyde : ', clyde.crs)

Trainlines:  {'init': 'epsg:27700'}
Stations :  {'init': 'epsg:27700'}
Subway :  {'init': 'epsg:27700'}
Clyde :  {'init': 'epsg:27700'}


In [11]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [12]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )

In [24]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

**Let’s now apply the functions that we have created and parse the x and y coordinates for all of our datasets.**

In [14]:
# Get the Polygon x and y coordinates
#grid['x'] = grid.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
#grid['y'] = grid.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

# Calculate x and y coordinates of the line
trainlines['x'] = trainlines.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
trainlines['y'] = trainlines.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)


clyde['x'] = clyde.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
clyde['y'] = clyde.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

In [25]:
# Calculate x and y coordinates of the points
subway['x'] = subway.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
subway['y'] = subway.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

stations['x'] = stations.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
stations['y'] = stations.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

**Let’s now convert our GeoDataFrames into Bokeh ColumnDataSources (without geometry columns)**

In [26]:
# Make a copy, drop the geometry column and create ColumnDataSource
tr_df = trainlines.drop('geometry', axis=1).copy()
trsource = ColumnDataSource(tr_df)

# Make a copy, drop the geometry column and create ColumnDataSource
cl_df = clyde.drop('geometry', axis=1).copy()
clsource = ColumnDataSource(cl_df)


# Make a copy, drop the geometry column and create ColumnDataSource
s_df = subway.drop('geometry', axis=1).copy()
ssource = ColumnDataSource(s_df)

# Make a copy, drop the geometry column and create ColumnDataSource
st_df = stations.drop('geometry', axis=1).copy()
stsource = ColumnDataSource(st_df)

**Data visualising**

In [42]:
# Initialize our figure
p = figure(title="Stations Glasgow",sizing_mode='scale_width')

# Add clyde on top of the same figure
p.multi_line('x', 'y', source=clsource, color="#98bcde", line_width=10, line_cap ='round', line_join = 'round')

# Add trainlines on top of the same figure
p.multi_line('x', 'y', source=trsource, color="#B0B0B0", line_width=4, line_cap ='round', line_join = 'round')
# Add subway on top (as black points)
subway = p.circle('x', 'y', size=10, source=ssource, color="lime")
# Add stations on top (as blue hollow points)
stations = p.circle('x', 'y', size=10, source=stsource,  line_color="#3288bd", fill_color="white", line_width=3)




In [43]:
# Hover tool referring to our own data field using @ and
## a position on the graph using $
# adding hover only to subway and railway stations
hover = HoverTool( tooltips = [('Name ', '@name')]) #,('(x,y)', '($x, $y)')])

p.add_tools(hover)
p.hover.renderers=[subway, stations]


In [46]:
#removing grid lines

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

In [48]:
#hiding axis
p.axis.visible = False

In [49]:
show(p)